In [1]:
from dataset.reflacx import REFLACXDataset
from torch.utils.data import DataLoader
import torch


# import warnings
# import pandas as pd
# pd.options.mode.chained_assignment = None  # default='warn'
# warnings.filterwarnings("ignore")

In [2]:
d = REFLACXDataset(
    df_path="./spreadsheets/reflacx_clinical.csv",
    mimic_eye_path="F:\\mimic-eye",
    image_size=128,
    split_str="train",
)

In [3]:
loader = DataLoader(
    d,
    batch_size=4,
    shuffle=True,
    collate_fn= lambda x: tuple(zip(*x)),
    drop_last=False,
)

In [4]:
data = next(iter(loader))

c:\Users\mike8\OneDrive\文件\GitHub\fusion-experiments\dataset\reflacx.py:205: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.tensor(data[self.chexpert_label_cols]) == 1
c:\Users\mike8\OneDrive\文件\GitHub\fusion-experiments\dataset\reflacx.py:223: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return torch.tensor(data[self.clinical_label])
c:\Users\mike8\OneDrive\文件\GitHub\fusion-experiments\dataset\reflacx.py:205: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a valu

In [5]:
data[0][0].keys()

dict_keys(['xray', 'bb_label', 'clinical_data', 'chexpert_label', 'report'])

In [6]:
modalities = ['xray', 'bb_label', 'clinical_data', 'chexpert_label', 'report']

batch_map = {
    m: [ d[m] for d in data[0]]
    for m in modalities
}

In [7]:
imgs = torch.stack(batch_map['xray'])

In [8]:
imgs.shape

torch.Size([4, 3, 128, 128])

In [9]:
hidden_dim = 128

In [10]:
# n = 1 # batch_size
n, c, h, w = imgs.shape

In [11]:
from torch import nn

patch_size = 8

conv_proj = nn.Conv2d(
    in_channels=3,
    out_channels=hidden_dim,
    kernel_size=patch_size,
    stride=patch_size,
)

In [12]:
n_h = h // patch_size
n_w = w // patch_size

In [13]:
img_seq  = conv_proj(imgs)

In [14]:
img_inputs = img_seq.reshape(n, hidden_dim, n_h*n_w)

(tensor([-0.5405], dtype=torch.float64),
 tensor([0.6974], dtype=torch.float64),
 tensor([-0.8318], dtype=torch.float64),
 tensor([0.1149], dtype=torch.float64))

In [17]:
bb_label = batch_map['bb_label']

In [18]:
bb_label_emb_dim = 32

In [19]:
bb_embedding_layer = nn.Embedding(
    num_embeddings=len(loader.dataset.label_cols)+1,
    embedding_dim=bb_label_emb_dim,
    padding_idx=0,
)
bb_fc = nn.Linear(4+bb_label_emb_dim, 128)

In [20]:
bb_inputs = []
for d in bb_label:
    if len(d["boxes"]) > 0:
        emb_l = bb_embedding_layer(d["labels"])
        bbs = bb_fc(torch.concat([d["boxes"], emb_l], dim=1))
        bb_inputs.append(bbs)
    else:
        bb_inputs.append(None)

In [21]:
clinical_cat_emb = nn.Embedding(2,  32)

In [23]:
clinical = batch_map['clinical_data']

In [24]:
clinical

[{'cat': {'gender': tensor(1, dtype=torch.int32)}, 'num': tensor([-0.4286])},
 {'cat': {'gender': tensor(0, dtype=torch.int32)}, 'num': tensor([0.7073])},
 {'cat': {'gender': tensor(0, dtype=torch.int32)}, 'num': tensor([-0.2123])},
 {'cat': {'gender': tensor(1, dtype=torch.int32)}, 'num': tensor([-0.7532])}]

In [25]:
clinical_map = {
    "cat": {},
}
for c in clinical:
    for k in c["cat"].keys():
        if not k in clinical_map["cat"]:
            clinical_map["cat"][k] = []
        clinical_map["cat"][k].append(c["cat"][k])

clinical_map["num"] = [c["num"] for c in clinical]

In [26]:
clinical_cat_emb = {
    "gender": nn.Embedding(2, 32),
}



In [27]:
clinical_cat = clinical_cat_emb['gender'](torch.stack(clinical_map["cat"]["gender"]))


In [28]:
clinical_num = torch.stack(clinical_map['num'])

In [29]:
clinical_fc = nn.Linear(33, 128)
clinical_inputs = clinical_fc(torch.concat([clinical_cat,clinical_num],dim=1))

In [30]:
clinical_inputs = clinical_inputs.unsqueeze(1)

In [32]:
chesxpert_label = batch_map['chexpert_label']

In [34]:
chexpert_fc = nn.Linear(14, 128)
chexpert_inputs= chexpert_fc(torch.stack(chesxpert_label).float()).unsqueeze(1)

In [35]:
chexpert_inputs.shape

torch.Size([4, 1, 128])

In [36]:
import torch
import math
from transformers import BioGptTokenizer


class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: torch.Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")
token_emb = TokenEmbedding(tokenizer.vocab_size, 128)


In [38]:
inputs = tokenizer(
    batch_map['report'],
    # return_tensors="pt",
)

In [39]:
report_inputs = [token_emb(torch.tensor(n)) for n in inputs['input_ids']]
    

In [40]:
report_inputs[0].shape

torch.Size([102, 128])

In [41]:
report_inputs[1].shape

torch.Size([111, 128])

In [42]:
img_inputs.shape

torch.Size([4, 128, 256])

In [43]:
bb_inputs

[None,
 None,
 None,
 tensor([[ 5.9379e-01, -5.3448e-01, -6.2671e-01, -4.8388e-01, -6.7895e-01,
           1.4315e-01, -1.5155e+00,  6.2987e-01, -9.9504e-03, -1.5155e-01,
          -9.1749e-03,  1.7801e-01,  1.4580e-01, -7.1542e-01,  1.5403e-02,
          -1.9494e-01,  4.9911e-01, -7.2490e-02,  3.1341e-01,  1.0999e+00,
          -9.6446e-02, -4.6743e-01, -8.9039e-01, -4.0554e-01,  3.6663e-01,
           2.1572e-01, -1.1009e-02, -3.7145e-01, -2.8702e-01, -3.4386e-01,
          -4.9316e-01,  4.3988e-01,  3.0049e-01,  3.8851e-01,  1.7379e-01,
           1.2000e-01, -6.3587e-01,  3.5331e-01,  5.3285e-01, -2.7295e-01,
           1.5249e-01,  9.3246e-01, -4.1730e-01,  5.9662e-01,  8.3072e-01,
           3.3380e-01, -9.6957e-01,  3.3031e-01, -1.1813e-01,  1.4225e-01,
           7.4860e-01,  3.9765e-03,  1.5640e-01, -4.9435e-01, -4.1868e-01,
          -6.5998e-01, -7.2326e-02,  2.5672e-02, -1.0318e+00,  3.5936e-01,
          -3.1958e-01,  1.3043e-03,  1.0909e-01,  2.9044e-01,  7.6512e-02,
    

In [38]:
clinical_inputs.shape

torch.Size([4, 1, 128])

In [39]:
chexpert_inputs.shape

torch.Size([4, 1, 128])

In [40]:
report_inputs

[tensor([[ 13.9371,  -1.0265,   3.0123,  ...,   4.0277,   1.7066, -20.8931],
         [  4.7147,  -3.2481, -14.8378,  ...,  -0.1010,  -7.4070,   2.5657],
         [ -0.7778,   6.3151,   1.2750,  ...,   4.0703, -10.1901, -14.5043],
         ...,
         [  1.5462,  10.8573,   6.5433,  ...,  -1.2977, -12.6543,   5.8778],
         [ 15.1477, -17.3649,  -7.9275,  ...,  17.5019,  -6.6169,   4.3734],
         [ 25.8858,   4.3370, -10.0249,  ...,   9.7746,   8.9536,  10.3916]],
        grad_fn=<MulBackward0>),
 tensor([[ 13.9371,  -1.0265,   3.0123,  ...,   4.0277,   1.7066, -20.8931],
         [  4.7147,  -3.2481, -14.8378,  ...,  -0.1010,  -7.4070,   2.5657],
         [ -0.7778,   6.3151,   1.2750,  ...,   4.0703, -10.1901, -14.5043],
         ...,
         [ -5.5855,  -4.6550,  -7.6614,  ...,   9.6503, -11.5157,   9.9651],
         [-19.5617,   1.5089, -10.5035,  ...,   8.4118, -14.7911,   3.0680],
         [ 25.8858,   4.3370, -10.0249,  ...,   9.7746,   8.9536,  10.3916]],
        grad_

In [44]:
bos_tensor = token_emb(torch.tensor([tokenizer.bos_token_id])).reshape(-1, 1)
eos_tensor = token_emb(torch.tensor([tokenizer.eos_token_id])).reshape(-1, 1)
pad_tensor = token_emb(torch.tensor([tokenizer.pad_token_id])).reshape(-1, 1)


In [45]:
input_list = []
for img, bb, c, chexpert, report in zip(
    img_inputs, bb_inputs, clinical_inputs, chexpert_inputs, report_inputs
):
    cat_list = [
        bos_tensor,
        img,
        eos_tensor,
        bos_tensor,
        c.reshape(hidden_dim, -1),
        eos_tensor,
        bos_tensor,
        chexpert.reshape(hidden_dim, -1),
        eos_tensor,
        bos_tensor,
        report.permute(1, 0),
        eos_tensor,
    ]

    if not bb is None and len(bb) > 0:
        cat_list += [
            bos_tensor,
            bb.permute(1, 0),
            eos_tensor,
        ]

    cat_t = torch.concat(cat_list, dim=1)
    input_list.append(cat_t)

In [46]:
padded_inputs = []

max_len = max([ i.shape[-1] for i in input_list])
for i in input_list:
    if i.shape[-1] < max_len:
        padded_inputs.append(torch.concat([i, pad_tensor.repeat(1, max_len-i.shape[-1])], dim=1))
    else:
        padded_inputs.append(i)

In [47]:
cat_inputs = torch.stack(padded_inputs)

In [48]:
cat_inputs = cat_inputs.permute(0, 2, 1)

In [49]:
cat_inputs.shape

torch.Size([4, 382, 128])

In [50]:
layer = nn.TransformerEncoderLayer(
    d_model=128,
    nhead=4,
    batch_first=True,
)
model = nn.TransformerEncoder(layer, num_layers=4, enable_nested_tensor=True)

In [51]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [52]:
pos_enc = PositionalEncoding(hidden_dim, dropout=0)

In [53]:
# outputs = model(torch.nested.nested_tensor(input_list)) # inference only

In [54]:
outputs = model(pos_enc(cat_inputs))

In [55]:
import torch

In [56]:
torch.tensor([0]*50+[1]*20)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [57]:
# take the last one and 

In [58]:
outputs[:, -1 , :].shape

torch.Size([4, 128])

In [59]:
output_proj =nn.Linear(hidden_dim, 1)

In [60]:
out = output_proj(outputs[:, -1 , :])

In [61]:
out.shape

torch.Size([4, 1])

In [69]:
a=[1,1,2]

In [70]:
torch.rand((3,3))[:, a,:]

IndexError: too many indices for tensor of dimension 2

In [99]:
a = torch.randn(3,3,3)

In [100]:
idx= torch.tensor([1, 2,1]).long()

In [102]:
a[:, idx, torch.arange(a.shape[-1])].shape

torch.Size([3, 3])

In [84]:
torch.index_select(a, 1, torch.tensor([1,2])).shape

torch.Size([3, 2, 3])

In [62]:
out

tensor([[-0.3337],
        [ 0.1281],
        [ 0.3768],
        [-0.8121]], grad_fn=<AddmmBackward0>)

In [72]:
torch.stack(data[-1])

tensor([[ 0.5518],
        [-0.3220],
        [-0.5405],
        [-0.1764]], dtype=torch.float64)

In [76]:
loss_fn = nn.MSELoss()

In [77]:
loss_fn(out, torch.stack(data[-1]))

tensor(0.2733, dtype=torch.float64, grad_fn=<MseLossBackward0>)